In [1]:
#Path to Images
impath = "images_training_rev1/"
#Path to Labels CSV
lapath = "training_solutions_rev1.csv"

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf
%matplotlib inline

In [3]:
label_df = pd.read_csv(lapath,index_col=0)
labels = label_df.to_numpy()
ids = label_df.index.to_numpy()
image_set = tf.keras.preprocessing.image_dataset_from_directory(impath,labels=labels,label_mode='categorical')

/Users/humzaqureshi/miniforge3/lib/python3.9/site-packages/keras/utils/image_dataset.py:156: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if labels not in ('inferred', None):


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()